# title

In [1]:
import time
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

In [2]:
def get_video_urls(youtube_playlist_url):
    #webdriver setup & html scrape
    driver = webdriver.Chrome('resources/chromedriver')
    driver.get(youtube_playlist_url)
    soup = BeautifulSoup(driver.page_source)

    #isolate the video stats
    video_stats = soup.find('div', id='stats').text
    #use regex to isolate video number
    no_of_videos = re.search('(\d{1,3},?\d{1,3},?)', video_stats).group()
    #number of scroll based on 100 thumbnails per scroll
    no_of_scrolls = int(int(no_of_videos) / 100) + 1

    #scroll page for every 100 videos
    for i in range(no_of_scrolls):
        driver.execute_script("window.scrollBy(0, 12000);")
        time.sleep(3)

    soup = BeautifulSoup(driver.page_source)
    driver.quit()

    #extract youtube playlist urls
    matches = soup.find_all('a', class_='yt-simple-endpoint style-scope ytd-playlist-video-renderer')
    youtube_urls = []
    for url in matches:
        string = url.get('href')
        substring = re.search(r'\/watch\?v=\w*', string).group()
        youtube_urls.append('http://www.youtube.com' + substring)    
    return youtube_urls

In [3]:
video = 'https://www.youtube.com/watch?v=4GokR2C9wy0'

- title
- url
- account name
- subscribers
- views
- date uploaded
- number of likes
- number of dislikes

In [76]:
def get_stats(youtube_url):
    driver = webdriver.Chrome('resources/chromedriver')
    driver.get(youtube_url)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source)
    driver.quit()
    
    def km_cleaner(value):
        if "subscribers" in value:
            if 'K' in value:
                value = float(re.sub('K subscribers', '', value))
                value *= 1000
                return int(value)
            elif 'M' in value:
                value = float(re.sub('M subscribers', '', value))
                value *= 1000000
                return int(value)
            else:
                value = float(re.sub('M subscribers', '', value))
                return int(value)
        else:
            if 'K' in value:
                value = float(re.sub('K', '', value))
                value *= 1000
                return int(value)
            elif 'M' in value:
                value = float(re.sub('M', '', value))
                value *= 1000000
                return int(value)
            else:
                return int(value)
    title_value = soup.find_all('yt-formatted-string')[3].text
    
    account_value = soup.find_all('yt-formatted-string')[9].text
    
    subs_string = soup.find_all('yt-formatted-string')[10].text
    subs_value = km_cleaner(subs_string)
    
    view_count_string = soup.find('yt-view-count-renderer').text
    view_count = re.search(r'(\d{1,3},?\d{1,3},?\d{1,3},?\d{1,3},?)', view_count_string).group().replace(',','')
    
    date_value = soup.find_all('yt-formatted-string')[4].text
    
    likes_string = soup.find_all('yt-formatted-string')[5].text
    likes_value = km_cleaner(likes_string)

    dislikes_string = soup.find_all('yt-formatted-string')[6].text
    dislikes_value = km_cleaner(dislikes_string)
    
    stats = {'url': youtube_url,
             'date_uploaded': date_value,
             'views': view_count,
             'account': account_value,
             'video_title': title_value,
             'subscribers': subs_value,
             'likes':likes_value,
             'dislikes':dislikes_value
            }
    
    return stats 

In [67]:
driver = webdriver.Chrome('resources/chromedriver')
driver.get(video)
time.sleep(3)
soup = BeautifulSoup(driver.page_source)
driver.quit()

In [57]:
def km_cleaner(value):
    if "subscribers" in value:
        if 'K' in value:
            value = float(re.sub('K subscribers', '', value))
            value *= 1000
            return int(value)
        elif 'M' in value:
            value = float(re.sub('M subscribers', '', value))
            value *= 1000000
            return int(value)
        else:
            value = float(re.sub('M subscribers', '', value))
            return int(value)
    else:
        if 'K' in value:
            value = float(re.sub('K', '', value))
            value *= 1000
            return int(value)
        elif 'M' in value:
            value = float(re.sub('M', '', value))
            value *= 1000000
            return int(value)
        else:
            return int(value)


In [8]:
#title
title_value = soup.find_all('yt-formatted-string')[3].text

In [46]:
#account name
account_value = soup.find_all('yt-formatted-string')[9].text

In [58]:
#subscribers
subs_string = soup.find_all('yt-formatted-string')[10].text
subs_value = km_cleaner(subs_string)

In [5]:
#viewcount
view_count_string = soup.find('yt-view-count-renderer').text
view_count = re.search(r'(\d{1,3},?\d{1,3},?\d{1,3},?\d{1,3},?)', view_count_string).group().replace(',','')

In [50]:
#date
date_string = soup.find_all('yt-formatted-string')[4].text
date_value = datetime.strptime(date_string, "%b %d, %Y")

In [60]:
likes_string = soup.find_all('yt-formatted-string')[5].text
likes_value = km_cleaner(likes_string)

dislikes_string = soup.find_all('yt-formatted-string')[6].text
dislikes_value = km_cleaner(dislikes_string)

In [77]:
get_stats("https://www.youtube.com/watch?v=H8jL9uRhIbI")

{'url': 'https://www.youtube.com/watch?v=H8jL9uRhIbI',
 'date_uploaded': 'Dec 16, 2008',
 'views': '267041',
 'account': 'KQED',
 'video_title': 'Go Fish (211): Jacques Pépin: More Fast Food My Way',
 'subscribers': 80000,
 'likes': 1300,
 'dislikes': 42}